<a href="https://colab.research.google.com/github/abdullahzunorain/About--RAG/blob/main/GEN_AI_RAG_DEMO_(module_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intall library in order to use the api for the model


In [ ]:
!pip install groq
!pip install sentence_transformers
!pip install streamlit

In [ ]:
# !pip install pandas faiss-cpu torch transformers


Get an api from the groq

Link To get Api Key: https://console.groq.com/keys

In [ ]:
# api_key="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"

In [ ]:
from google.colab import userdata
GROQ_API_TOKEN = userdata.get('GROQ_API_TOKEN')

Choose model of your choice

Link for getting Models: https://console.groq.com/docs

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=GROQ_API_TOKEN,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "what is ai",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Find a Dataset on which u want to make a RAG

Link for the dataset: https://www.kaggle.com/datasets/utkarshx27/movies-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("utkarshx27/movies-dataset")

print("Path to dataset files:", path)

In [ ]:
!pip install streamlit
# !pip install faiss
!pip install faiss-gpu

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
# !streamlit run app.py & npx localtunnel --port 8501

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import kagglehub
import os

In [ ]:
# Download latest version
path = kagglehub.dataset_download("utkarshx27/movies-dataset")
print("Path to dataset files:", path)

# Find the CSV file within the downloaded directory
# Assuming there's only one CSV file in the directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        csv_file = os.path.join(path, filename)
        break  # Stop searching after finding the first CSV file
else:
    raise FileNotFoundError("No CSV file found in the downloaded dataset directory.")

# Load CSV data
df = pd.read_csv(csv_file) # Assuming the file is a standard CSV


In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Use 'overview' as the source for documents
documents = df['overview'].tolist()

# Clean the documents list to ensure all entries are strings
documents = [str(doc) if pd.notnull(doc) else "" for doc in documents]

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each document
doc_embeddings = model.encode(documents, show_progress_bar=True)

# Convert to NumPy array (FAISS requires float32)
embedding_matrix = np.array(doc_embeddings).astype("float32")

# Build FAISS index for efficient similarity search
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Function to retrieve the most relevant document
def retrieve(query, top_k=1):
    query_embedding = model.encode(query)  # Encode the query
    query_vector = np.array(query_embedding).astype("float32")

    # Search for the closest document in the FAISS index
    distances, indices = index.search(np.array([query_vector]), top_k)
    return [documents[idx] for idx in indices[0]]

# Function to generate response based on retrieved document
def generate_response(query):
    relevant_docs = retrieve(query)
    input_text = f"{relevant_docs[0]} Context: {query}"

    return input_text  # Simply return the relevant document for demonstration

# Test RAG with a sample query
query = "What are some interesting movies from the last year?"
response = generate_response(query)
print("Generated Response:", response)